In [ ]:
#importing all the modules needed fo running the script 
import os 
import numpy as np
import librosa
import matplotlib.pyplot as plt
import random
import scipy.io.wavfile as wav

In [ ]:
# DATASET LOADING PARAMETERS
case = 1
# 'dev' - development or 'eval' - evaluation dataset, or both. 
process_str = 'dev, eval'

# 'dev' - development or 'eval' - evaluation dataset
mode='eval'
# 'foa' - ambisonic or 'mic' - microphone signals                                               
dataset='foa'    

   
dataset_aug = False if dataset == 'mic' else True


base_dir = 'Base directory'
#defining cross-validation split
if mode == 'dev':
  train_splits = [3, 4, 5, 6]
elif mode == 'eval':
  train_splits = [2, 3, 4, 5, 6]

#how many new files we want to create from the original one.
data_augmentation_nb = 2


dataset_dir = base_dir + 'Dataset'

    
feat_label_dir= os.path.join(dataset_dir, 'feat_label-AR')       

image_dir = os.path.join(base_dir, 'images') 

#pattern rotation to consider for data augmentation. 

rotation_pattern = 15
                                                                                                                                                                                      
#FEATURE PARAMS
fs=24000
hop_len_s=0.02
label_hop_len_s=0.1
max_audio_len_s=60
nb_mel_bins=64            

unique_classes = {
            'alarm': 0,
            'baby': 1,
            'crash': 2,
            'dog': 3,
            'engine': 4,
            'female_scream': 5,
            'female_speech': 6,
            'fire': 7,
            'footsteps': 8,
            'knock': 9,
            'male_scream': 10,
            'male_speech': 11,
            'phone': 12,
            'piano': 13
        }


    # ########### User defined parameters ##############
    
if case == 1:
  print("USING DEFAULT PARAMETERS\n")

elif case == 2:
  mode = 'dev'
  dataset = 'mic'

elif case == 3:
   mode = 'eval'
   dataset = 'mic'

elif case == 4:
    mode = 'dev'
    dataset = 'foa'

elif case == 5:
    mode = 'eval'
    dataset = 'foa'

elif case == 999:
      print("QUICK TEST MODE\n")
      quick_test = True
      epochs_per_fit = 1

else:
    print('ERROR: unknown argument {}'.format(case))
    exit()
       

In [ ]:
#function to create a new folder in case it does not exist yet. 
def create_folder(folder_name):
    
    if not os.path.exists(folder_name):
        print('{} folder does not exist, creating it.'.format(folder_name))
        os.makedirs(folder_name)

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import os
import random
import scipy.io.wavfile as wav


def apply_data_augmentation(audio):
    
    
    #selecting the a random pattern between the 15 implemented
    pattern = random.randrange(rotation_pattern) # original case not considered for augmentation
    print("Data augmentation pattern {}".format(pattern))
        
    audio_aug = np.zeros(audio.shape)

    w = audio[:, 0]
    y = audio[:, 1]
    z = audio[:, 2]
    x = audio[:, 3]
    
    # w channel never change 
    audio_aug[:, 0] = w

    # the 15 pattern rotations and reflection
    if (pattern == 0):
        #print('Φ - pi/2')
        audio_aug[:, 1] = -x
        audio_aug[:, 3] = y
        audio_aug[:, 2] = z  
    elif (pattern == 1):
        #print('Φ - pi/2, -θ')
        audio_aug[:, 1] = -x
        audio_aug[:, 3] = y
        audio_aug[:, 2] = -z
    
    elif (pattern == 2):
        #print('Φ, -θ')
        audio_aug[:, 1] = y
        audio_aug[:, 3] = x
        audio_aug[:, 2] = -z
    
    elif (pattern == 3):
        #print('Φ + pi/2')
        audio_aug[:, 1] = x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = z    
    elif (pattern == 4):    
        #print('Φ + pi/2, -θ')
        audio_aug[:, 1] = x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = -z
        
    elif (pattern == 5):
        #print('-Φ - pi/2')
        audio_aug[:, 1] = x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = z
    elif (pattern == 6):
        #print('-Φ - pi/2, -θ')
        audio_aug[:, 1] = -x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = -z
    
    elif (pattern == 7):
        #print('-Φ')
        audio_aug[:, 1] = -y
        audio_aug[:, 3] = x
        audio_aug[:, 2] = z  
    elif (pattern == 8):
        
        
        audio_aug[:, 1] = -y
        audio_aug[:, 3] = x
        audio_aug[:, 2] = -z
    
    elif (pattern == 9):
        
        audio_aug[:, 1] = x
        audio_aug[:, 3] = y
        audio_aug[:, 2] = z  
    elif (pattern == 10):
       
        audio_aug[:, 1] = x
        audio_aug[:, 3] = y
        audio_aug[:, 2] = -z
    
    elif (pattern == 11):
       
        audio_aug[:, 1] = y
        audio_aug[:, 3] = -x
        audio_aug[:, 2] = z      
    elif (pattern == 12):
        
        audio_aug[:, 1] = y
        audio_aug[:, 3] = -x
        audio_aug[:, 2] = -z
        
    elif (pattern == 13):
       
        audio_aug[:, 1] = -x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = z    
    elif (pattern == 14):
        
        audio_aug[:, 1] = -x
        audio_aug[:, 3] = -y
        audio_aug[:, 2] = -z  
    
    else:
        print("Wrong pattern selection")
        
    return audio_aug, pattern


def label_rotation(label, pattern):

  

  label_aug = np.zeros(len(label))
    # w channel never change 
  label_aug[0] = label[0]

  if (pattern == 0):
        #print('Φ - pi/2')
        label_aug[1] = label[2]
        label_aug[2] = -label[1]
        label_aug[3] = label[3]
  elif (pattern == 1):
        #print('Φ - pi/2, -θ')
        label_aug[1] = label[2]
        label_aug[2] = -label[1]
        label_aug[3] = -label[3]
    
  elif (pattern == 2):
        #print('Φ, -θ')
        label_aug[1] = label[1]
        label_aug[2] = label[2]
        label_aug[3] = -label[3]
    
  elif (pattern == 3):
        #print('Φ + pi/2')
        label_aug[1] = -label[2]
        label_aug[2] = label[1]
        label_aug[3] = label[3]
  elif (pattern == 4):    
        #print('Φ + pi/2, -θ')
        label_aug[1] = -label[2]
        label_aug[2] = label[1]
        label_aug[3] = -label[3]
        
  elif (pattern == 5):
        #print('-Φ - pi/2')
        label_aug[1] = -label[2]
        label_aug[2] = label[1]
        label_aug[3] = label[3]
  elif (pattern == 6):
        #print('-Φ - pi/2, -θ')
        label_aug[1] = -label[2]
        label_aug[2] = -label[1]
        label_aug[3] = -label[3]
    
  elif (pattern == 7):
        #print('-Φ')
        label_aug[1] = label[1]
        label_aug[2] = -label[2]
        label_aug[3] = label[3]
  elif (pattern == 8):
        # - azimuth - pi/2 - elevation
        #print('-Φ, -θ')
        label_aug[1] = label[1]
        label_aug[2] = -label[2]
        label_aug[3] = -label[3]
    
  elif (pattern == 9):
        #print('-Φ + pi/2')
        label_aug[1] = label[2]
        label_aug[2] = label[1]
        label_aug[3] = label[3]
  elif (pattern == 10):
        #print('-Φ + pi/2, -θ')
        label_aug[1] = label[2]
        label_aug[2] = label[1]
        label_aug[3] = -label[3]
    
  elif (pattern == 11):
        #print('-Φ + pi')
        label_aug[1] = -label[1]
        label_aug[2] = label[2]
        label_aug[3] = label[3]     
  elif (pattern == 12):
        # azimuth + pi/2 - elevation
        #print('-Φ + pi, -θ')
        label_aug[1] = -label[1]
        label_aug[2] = label[2]
        label_aug[3] = -label[3]
        
  elif (pattern == 13):
        #print('Φ + pi')
        label_aug[1] = -label[2]
        label_aug[2] = -label[1]
        label_aug[3] = label[3]
  elif (pattern == 14):
        #print('Φ + pi, -θ')
        label_aug[1] = -label[2]
        label_aug[2] = -label[1]
        label_aug[3] = -label[3]
    
  else:
        print("Wrong pattern selection")
        
  return label_aug

def label_augmentation(label_dir, pattern):

    #print("Label augmentation pattern {}".format(pattern))
    
    

    label_aug_dict = {}
    for frame_cnt in label_dir.keys():
        if frame_cnt not in label_aug_dict:
            label_aug_dict[frame_cnt] = []
            for tmp_val in label_dir[frame_cnt]:
                aug_tmp_value = label_rotation(tmp_val, pattern)
                label_aug_dict[frame_cnt].append(aug_tmp_value)
    return label_aug_dict

In [ ]:
# Contains routines for labels creation, features extraction and normalization
from sklearn import preprocessing
import joblib
import math
import matplotlib.pyplot as plot


def nCr(n, r):
    return math.factorial(n) // math.factorial(r) // math.factorial(n-r)


class FeatureClass:
    def __init__(self, is_eval=False):
        

        # Input directories
        self._feat_label_dir = feat_label_dir
        self._dataset_dir = dataset_dir
        self._dataset_combination = '{}_{}'.format(dataset, 'eval' if is_eval else 'dev')
        
        self._aud_dir = os.path.join(self._dataset_dir, self._dataset_combination)
        self._desc_dir = None if is_eval else os.path.join(self._dataset_dir, "metadata_dev")

        # Output directories
        self._label_dir = None
        self._feat_dir = None
        self._feat_dir_norm = None

        # Local parameters
        self._is_eval = is_eval

        self._fs = fs
        self._hop_len_s = hop_len_s
        self._hop_len = int(self._fs * self._hop_len_s)

        self._label_hop_len_s = label_hop_len_s
        self._label_hop_len = int(self._fs * self._label_hop_len_s)
        self._label_frame_res = self._fs / float(self._label_hop_len)
        self._nb_label_frames_1s = int(self._label_frame_res)

        self._win_len = 2 * self._hop_len
        self._nfft = self._next_greater_power_of_2(self._win_len)
        self._nb_mel_bins = nb_mel_bins
        self._mel_wts = librosa.filters.mel(sr=self._fs, n_fft=self._nfft, n_mels=self._nb_mel_bins).T

        self._dataset = dataset
        self._eps = 1e-8
        self._nb_channels = 4

        # Sound event classes dictionary
        self._unique_classes = unique_classes
        self._audio_max_len_samples = max_audio_len_s * self._fs 

        self._max_feat_frames = int(np.ceil(self._audio_max_len_samples / float(self._hop_len)))
        self._max_label_frames = int(np.ceil(self._audio_max_len_samples / float(self._label_hop_len)))

    def _load_audio(self, audio_path):

        fs, audio = wav.read(audio_path)
        audio = audio[:, :self._nb_channels] / 32768.0 + self._eps
        if audio.shape[0] < self._audio_max_len_samples:
            zero_pad = np.random.rand(self._audio_max_len_samples - audio.shape[0], audio.shape[1])*self._eps
            audio = np.vstack((audio, zero_pad))
        elif audio.shape[0] > self._audio_max_len_samples:
            audio = audio[:self._audio_max_len_samples, :]
        return audio, fs

    # INPUT FEATURES
    @staticmethod
    def _next_greater_power_of_2(x):
        return 2 ** (x - 1).bit_length()

    def _spectrogram(self, audio_input):
      
      _nb_ch = audio_input.shape[1]
      nb_bins = self._nfft // 2
      spectra = np.zeros((self._max_feat_frames, nb_bins + 1, _nb_ch), dtype=complex)
      for ch_cnt in range(_nb_ch):
          stft_ch = librosa.core.stft(np.asfortranarray(audio_input[:, ch_cnt]), n_fft=self._nfft, hop_length=self._hop_len,
                                        win_length=self._win_len, window='hann')
          spectra[:, :, ch_cnt] = stft_ch[:, :self._max_feat_frames].T
      return spectra

    def _get_mel_spectrogram(self, linear_spectra_list):
        
        
        mel_feat_list = []
        for spec in range(len(linear_spectra_list)):
            linear_spectra = linear_spectra_list[spec]
            mel_feat = np.zeros((linear_spectra.shape[0], self._nb_mel_bins, linear_spectra.shape[-1]))
            for ch_cnt in range(linear_spectra.shape[-1]):
                mag_spectra = np.abs(linear_spectra[:, :, ch_cnt])**2
                mel_spectra = np.dot(mag_spectra, self._mel_wts)
                log_mel_spectra = librosa.power_to_db(mel_spectra)
                mel_feat[:, :, ch_cnt] = log_mel_spectra
            mel_feat = mel_feat.reshape((linear_spectra.shape[0], self._nb_mel_bins * linear_spectra.shape[-1]))
            mel_feat_list.append(mel_feat)

        return mel_feat_list

    def _get_foa_intensity_vectors(self, linear_spectra_list):
       
        foa_iv_list = []
        for spec in range(len(linear_spectra_list)):
            linear_spectra = linear_spectra_list[spec]
            IVx = np.real(np.conj(linear_spectra[:, :, 0]) * linear_spectra[:, :, 1])
            IVy = np.real(np.conj(linear_spectra[:, :, 0]) * linear_spectra[:, :, 2])
            IVz = np.real(np.conj(linear_spectra[:, :, 0]) * linear_spectra[:, :, 3])

            normal = np.sqrt(IVx ** 2 + IVy ** 2 + IVz ** 2) + self._eps
            IVx = np.dot(IVx / normal, self._mel_wts)
            IVy = np.dot(IVy / normal, self._mel_wts)
            IVz = np.dot(IVz / normal, self._mel_wts)

            # we are doing the following instead of simply concatenating to keep the processing similar to mel_spec and gcc
            foa_iv = np.dstack((IVx, IVy, IVz))
            foa_iv = foa_iv.reshape((linear_spectra.shape[0], self._nb_mel_bins * 3))
            if np.isnan(foa_iv).any():
                print('Feature extraction is generating nan outputs')
                exit()
            foa_iv_list.append(foa_iv)

        return foa_iv_list



    def _get_gcc(self, linear_spectra_list):
        
        gcc_feat_list = []
        for linear_spectra in linear_spectra_list:
            gcc_channels = nCr(linear_spectra.shape[-1], 2)
            gcc_feat = np.zeros((linear_spectra.shape[0], self._nb_mel_bins, gcc_channels))
            cnt = 0
            for m in range(linear_spectra.shape[-1]):
                for n in range(m+1, linear_spectra.shape[-1]):
                    R = np.conj(linear_spectra[:, :, m]) * linear_spectra[:, :, n]
                    cc = np.fft.irfft(np.exp(1.j*np.angle(R)))
                    cc = np.concatenate((cc[:, -self._nb_mel_bins//2:], cc[:, :self._nb_mel_bins//2]), axis=-1)
                    gcc_feat[:, :, cnt] = cc
                    cnt += 1
            gcc_feat = gcc_feat.reshape((linear_spectra.shape[0], self._nb_mel_bins*gcc_channels))
            gcc_feat_list.append(gcc_feat)

        return gcc_feat_list

    def _get_spectrogram_for_file(self, audio_filename):
        
        list_spec = []
        
        if dataset == 'foa':
          pattern = np.zeros(data_augmentation_nb, dtype=int)

        audio_in, fs = self._load_audio(os.path.join(self._aud_dir, audio_filename))
        audio_spec = self._spectrogram(audio_in)

        list_spec.append(audio_spec)

        if audio_filename[4] in train_splits and dataset == 'foa' and not self._is_eval:
            for i in range(data_augmentation_nb):
                audio_aug, pattern[i] = apply_data_augmentation(audio_in)
                audio_spec_aug = self._spectrogram(audio_aug)
                list_spec.append(audio_spec_aug)

        if dataset == 'foa':
          return list_spec, pattern
        else:
          return list_spec


    # OUTPUT LABELS
    def get_labels_for_file(self, _desc_file):
        
        se_label = np.zeros((self._max_label_frames, len(self._unique_classes)))
        x_label = np.zeros((self._max_label_frames, len(self._unique_classes)))
        y_label = np.zeros((self._max_label_frames, len(self._unique_classes)))
        z_label = np.zeros((self._max_label_frames, len(self._unique_classes)))

        for frame_ind, active_event_list in _desc_file.items():
            if frame_ind < self._max_label_frames:
                for active_event in active_event_list:
                    se_label[frame_ind, active_event[0]] = 1
                    x_label[frame_ind, active_event[0]] = active_event[1]
                    y_label[frame_ind, active_event[0]] = active_event[2]
                    z_label[frame_ind, active_event[0]] = active_event[3]

        label_mat = np.concatenate((se_label, x_label, y_label, z_label), axis=1)
        return label_mat

    # ------------------------------- EXTRACT FEATURE AND PREPROCESS IT -------------------------------
    def extract_all_feature(self):
        
        # setting up folders
        self._feat_dir = self.get_unnormalized_feat_dir()
        create_folder(self._feat_dir)

        # extraction starts
        print('Extracting spectrogram:')
        print('\t\taud_dir {}\n\t\tdesc_dir {}\n\t\tfeat_dir {}'.format(
            self._aud_dir, self._desc_dir, self._feat_dir))
        for split in os.listdir(self._aud_dir):
            print('Split: {}'.format(split))
            for file_cnt, file_name in enumerate(os.listdir(os.path.join(self._aud_dir, split))):
#             for file_cnt, file_name in enumerate(os.listdir(self._aud_dir)):
                #save the pattern implemented for each file
                if file_name != '.DS_Store':
                    wav_filename = '{}.wav'.format(file_name.split('.')[0])
                    if dataset == 'foa':
                      # implementation of data augmnetation based on channel rotations
                       
                      spect, pattern = self._get_spectrogram_for_file(os.path.join(self._aud_dir, split, wav_filename))
                    else:
                      spect = self._get_spectrogram_for_file(os.path.join(self._aud_dir, split, wav_filename))

                    #extract mel
                    mel_spect = self._get_mel_spectrogram(spect)

                    feat_list = []
                    if self._dataset is 'foa':
                        # extract intensity vectors
                        foa_iv = self._get_foa_intensity_vectors(spect)
                        for foa_index in range(len(foa_iv)):
#                             plot figures
                            
#                             print("Plotting")
#                             plot.figure(figsize=(22, 5))

#                             plot.subplot(211, xlabel='Time', ylabel='Frequency', title='mel-spect %s' % (foa_index)), \
#                             plot.imshow(mel_spect[foa_index].T)


#                             plot.subplot(212, xlabel='Time', ylabel='Frequency', title='intensity vector'), \
#                             plot.imshow(foa_iv[foa_index].T)
#                             plot.savefig(fname=image_dir + '%s_plot_%s.png' % (file_name.split('.')[0], foa_index), format='png')
#                             plot.close()

                            feat = np.concatenate((mel_spect[foa_index], foa_iv[foa_index]), axis=-1)
                            feat_list.append(feat)

                    elif self._dataset is 'mic':
                        # extract gcc
                        gcc = self._get_gcc(spect)
                        for gcc_index in range(len(gcc)):

                            # plot figures
                            # print("Plotting")
                            #plot.figure(figsize=(22, 5))

                            #plot.subplot(211, xlabel='Time', ylabel='Frequency', title='mel-spect %s' % (gcc_index))
                            #plot.imshow(mel_spect[gcc_index].T)

                            #plot.subplot(212, xlabel='Time', ylabel='Frequency', title='generilized cross corrrelation')
                            #plot.imshow(gcc[gcc_index].T)
                            #plot.savefig(fname=image_dir + '%s_plot_%s.png' % (file_name.split('.')[0], gcc_index),
                             #            format='png')

                            feat = np.concatenate((mel_spect[gcc_index], gcc[gcc_index]), axis=-1)
                            feat_list.append(feat)
                    else:
                        print('ERROR: Unknown dataset format {}'.format(self._dataset))
                        exit()

                    if len(feat_list) != 0:
                        for element in range(len(feat_list)):
                            print('{}: {}, {}, {}'.format(file_cnt, file_name, feat_list[element].shape, element))
                            if element == 0:
                              #no need to change the label name 
                              np.save(os.path.join(self._feat_dir, '{}-{}.npy'.format(wav_filename.split('.')[0], element)), feat_list[element]) 
                            else:
                              #augmented data
                              np.save(os.path.join(self._feat_dir, '{}-{}-{}.npy'.format(wav_filename.split('.')[0], element, pattern[element-1])), feat_list[element])
                              #creation of augmented data labels
                              label_folder = self._desc_dir
                              label_path = os.path.join(label_folder, wav_filename.replace('.wav', '.csv'))
                              label_dir_polar = self.load_output_format_file(label_path)
                              label_dir = self.convert_output_format_polar_to_cartesian(label_dir_polar)
                              #make the same rotation for the label and save with the same name on the metadata folder 
                              label_aug_dict = label_augmentation(label_dir, pattern[element-1])
                              label_aug_dict_pol = self.convert_output_format_cartesian_to_polar(label_aug_dict)
                              output_file_path = os.path.join(label_folder, '{}-{}-{}.csv'.format(wav_filename.split('.')[0], element, pattern[element-1]))
                              self.write_polar_file(output_file_path, label_aug_dict_pol)
                          
                      


    def preprocess_features(self):
        
        # Setting up folders and filenames
        self._feat_dir = self.get_unnormalized_feat_dir()
        self._feat_dir_norm = self.get_normalized_feat_dir()
        create_folder(self._feat_dir_norm)
        normalized_features_wts_file = self.get_normalized_wts_file()
        spec_scaler = None

        # pre-processing starts
        if self._is_eval:
            spec_scaler = joblib.load(normalized_features_wts_file)
            print('Normalized_features_wts_file: {}. Loaded.'.format(normalized_features_wts_file))

        else:
            #normalization using partial fit only on training dataset
            print('Estimating weights for normalizing feature files:')
            print('\t\tfeat_dir: {}'.format(self._feat_dir))

            spec_scaler = preprocessing.StandardScaler()
            for file_cnt, file_name in enumerate(os.listdir(self._feat_dir)):
                print('{}: {}'.format(file_cnt, file_name))
                feat_file = np.load(os.path.join(self._feat_dir, file_name))
                spec_scaler.partial_fit(feat_file)
                del feat_file
            joblib.dump(
                spec_scaler,
                normalized_features_wts_file
            )
            print('Normalized_features_wts_file: {}. Saved.'.format(normalized_features_wts_file))

        #transformign all the dataset
        print('Normalizing feature files:')
        print('\t\tfeat_dir_norm {}'.format(self._feat_dir_norm))
        for file_cnt, file_name in enumerate(os.listdir(self._feat_dir)):
            print('{}: {}'.format(file_cnt, file_name))
            feat_file = np.load(os.path.join(self._feat_dir, file_name))
            feat_file = spec_scaler.transform(feat_file)
            np.save(
                os.path.join(self._feat_dir_norm, file_name),
                feat_file
            )
            del feat_file

        print('normalized files written to {}'.format(self._feat_dir_norm))

    # ------------------------------- EXTRACT LABELS AND PREPROCESS IT -------------------------------
#     def extract_all_labels(self):
#         """
#          The function properly extracts all the labels
#         """
#         self._label_dir = self.get_label_dir()

#         print('Extracting labels:')
#         print('\t\taud_dir {}\n\t\tdesc_dir {}\n\t\tlabel_dir {}'.format(
#             self._aud_dir, self._desc_dir, self._label_dir))
#         create_folder(self._label_dir)
#         for split in os.listdir(self._desc_dir):
#             print('Split: {}'.format(split))
#             for file_cnt, file_name in enumerate(os.listdir(os.path.join(self._desc_dir, split)):
#                 print(file_name)
#                 if len(file_name)!=26: #checking clean metadata files #TODO this is not required if the dataset is clean
#                     continue
#                 print("label >>",file_name)
#                 wav_filename = '{}.wav'.format(file_name.split('.')[0])
#                 desc_file_polar = self.load_output_format_file(os.path.join(self._desc_dir, file_name))
#                 desc_file = self.convert_output_format_polar_to_cartesian(desc_file_polar)
#                 label_mat = self.get_labels_for_file(desc_file)
#                 print('{}: {}, {}'.format(file_cnt, file_name, label_mat.shape))
#                 np.save(os.path.join(self._label_dir, '{}.npy'.format(wav_filename.split('.')[0])), label_mat)



    def extract_all_labels(self):
            self._label_dir = self.get_label_dir()

            print('Extracting labels:')
            print('\t\taud_dir {}\n\t\tdesc_dir {}\n\t\tlabel_dir {}'.format(
                self._aud_dir, self._desc_dir, self._label_dir))
            create_folder(self._label_dir)
            for split in os.listdir(self._desc_dir):
                split_dir = os.path.join(self._desc_dir, split)
                print('Split: {}'.format(split_dir))
                
                for file_cnt, file_name in enumerate(os.listdir(split_dir)):
                    wav_filename = '{}.wav'.format(file_name.split('.')[0])
                    
                    desc_file_polar = self.load_output_format_file(os.path.join(os.path.join(self._desc_dir, split, file_name)))
                    desc_file = self.convert_output_format_polar_to_cartesian(desc_file_polar)
                    
                    label_mat = self.get_labels_for_file(desc_file)
                    print('\t{}: {}, {}'.format(file_cnt, file_name, label_mat.shape))
                    np.save(os.path.join(self._label_dir, '{}.npy'.format(wav_filename.split('.')[0])), label_mat)

    # -------------------------------  DCASE OUTPUT  FORMAT FUNCTIONS -------------------------------
    def load_output_format_file(self, _output_format_file):
        
        _output_dict = {}
        _fid = open(_output_format_file, 'r')
        # next(_fid)
        for _line in _fid:
            _words = _line.strip().split(',')
            _frame_ind = _words[0]
            try:
                _frame_ind = int(_words[0])
            except:
                print ("error",_words[0])
            if _words[0] not in _output_dict:
                _output_dict[_frame_ind] = []
            if len(_words) == 5: #read polar coordinates format, we ignore the track count 
                _output_dict[_frame_ind].append([int(_words[1]), float(_words[3]), float(_words[4]), int(_words[2])])
            elif len(_words) == 6: # read Cartesian coordinates format, we ignore the track count
                _output_dict[_frame_ind].append([int(_words[1]), float(_words[3]), float(_words[4]), float(_words[5]), int(_words[2])])
        _fid.close()
        return _output_dict


    def write_output_format_file(self, _output_format_file, _output_format_dict):
        
        _fid = open(_output_format_file, 'w')
        # _fid.write('{},{},{},{}\n'.format('frame number with 20ms hop (int)', 'class index (int)', 'azimuth angle (int)', 'elevation angle (int)'))
        for _frame_ind in _output_format_dict.keys():
            for _value in _output_format_dict[_frame_ind]:
                # Write Cartesian format output. Since baseline does not estimate track count we use a fixed value.
                _fid.write('{},{},{},{},{},{}\n'.format(int(_frame_ind), int(_value[0]), 0, float(_value[1]), float(_value[2]), float(_value[3])))
        _fid.close()

    def write_polar_file(self, output_file, output_format_dict):
        
        _fid = open(output_file, 'w')
        # _fid.write('{},{},{},{}\n'.format('frame number with 20ms hop (int)', 'class index (int)', 'azimuth angle (int)', 'elevation angle (int)'))
        for _frame_ind in output_format_dict.keys():
            for _value in output_format_dict[_frame_ind]:
              # Write polar coordinates format output. Since baseline does not estimate track count we use a fixed value.
                _fid.write('{},{},{},{},{}\n'.format(int(_frame_ind), int(_value[0]), 0, int(_value[1]), int(_value[2])))
    
        _fid.close()

    def segment_labels(self, _pred_dict, _max_frames):
        
        nb_blocks = int(np.ceil(_max_frames/float(self._nb_label_frames_1s)))
        output_dict = {x: {} for x in range(nb_blocks)}
        for frame_cnt in range(0, _max_frames, self._nb_label_frames_1s):

            # Collect class-wise information for each block
            # [class][frame] = <list of doa values>
            # Data structure supports multi-instance occurence of same class
            block_cnt = frame_cnt // self._nb_label_frames_1s
            loc_dict = {}
            for audio_frame in range(frame_cnt, frame_cnt+self._nb_label_frames_1s):
                if audio_frame not in _pred_dict:
                    continue
                for value in _pred_dict[audio_frame]:
                    if value[0] not in loc_dict:
                        loc_dict[value[0]] = {}

                    block_frame = audio_frame - frame_cnt
                    if block_frame not in loc_dict[value[0]]:
                        loc_dict[value[0]][block_frame] = []
                    loc_dict[value[0]][block_frame].append(value[1:])

            # Update the block wise details collected above in a global structure
            for class_cnt in loc_dict:
                if class_cnt not in output_dict[block_cnt]:
                    output_dict[block_cnt][class_cnt] = []

                keys = [k for k in loc_dict[class_cnt]]
                values = [loc_dict[class_cnt][k] for k in loc_dict[class_cnt]]

                output_dict[block_cnt][class_cnt].append([keys, values])

        return output_dict

    def regression_label_format_to_output_format(self, _sed_labels, _doa_labels):
        
        _nb_classes = len(self._unique_classes)
        _is_polar = _doa_labels.shape[-1] == 2*_nb_classes
        _azi_labels, _ele_labels = None, None
        _x, _y, _z = None, None, None
        if _is_polar:
            _azi_labels = _doa_labels[:, :_nb_classes]
            _ele_labels = _doa_labels[:, _nb_classes:]
        else:
            _x = _doa_labels[:, :_nb_classes]
            _y = _doa_labels[:, _nb_classes:2*_nb_classes]
            _z = _doa_labels[:, 2*_nb_classes:]

        _output_dict = {}
        for _frame_ind in range(_sed_labels.shape[0]):
            _tmp_ind = np.where(_sed_labels[_frame_ind, :])
            if len(_tmp_ind[0]):
                _output_dict[_frame_ind] = []
                for _tmp_class in _tmp_ind[0]:
                    if _is_polar:
                        _output_dict[_frame_ind].append([_tmp_class, _azi_labels[_frame_ind, _tmp_class], _ele_labels[_frame_ind, _tmp_class]])
                    else:
                        _output_dict[_frame_ind].append([_tmp_class, _x[_frame_ind, _tmp_class], _y[_frame_ind, _tmp_class], _z[_frame_ind, _tmp_class]])
        return _output_dict

    def convert_output_format_polar_to_cartesian(self, in_dict):
        
        out_dict = {}
        for frame_cnt in in_dict.keys():
            if frame_cnt not in out_dict:
                out_dict[frame_cnt] = []
                for tmp_val in in_dict[frame_cnt]:

                    ele_rad = tmp_val[2]*np.pi/180.
                    azi_rad = tmp_val[1]*np.pi/180

                    tmp_label = np.cos(ele_rad)
                    x = np.cos(azi_rad) * tmp_label
                    y = np.sin(azi_rad) * tmp_label
                    z = np.sin(ele_rad)
                    out_dict[frame_cnt].append([tmp_val[0], x, y, z])
        return out_dict

    def convert_output_format_cartesian_to_polar(self, in_dict):
        
        out_dict = {}
        for frame_cnt in in_dict.keys():
            if frame_cnt not in out_dict:
                out_dict[frame_cnt] = []
                for tmp_val in in_dict[frame_cnt]:
                    x, y, z = tmp_val[1], tmp_val[2], tmp_val[3]

                    # in degrees
                    azimuth = np.arctan2(y, x) * 180 / np.pi
                    elevation = np.arctan2(z, np.sqrt(x**2 + y**2)) * 180 / np.pi
                    r = np.sqrt(x**2 + y**2 + z**2)
                    out_dict[frame_cnt].append([tmp_val[0], azimuth, elevation])
        return out_dict

    # ------------------------------- Misc public functions -------------------------------
    def get_classes(self):
        
        return self._unique_classes

    def get_normalized_feat_dir(self):
        
        return os.path.join(
            self._feat_label_dir,
            '{}_norm'.format(self._dataset_combination)
        )

    def get_unnormalized_feat_dir(self):
        
        return os.path.join(
            self._feat_label_dir,
            '{}'.format(self._dataset_combination)
        )

    def get_label_dir(self):
        
        if self._is_eval:
            return None
        else:
            return os.path.join(
                self._feat_label_dir, '{}_label'.format(self._dataset_combination)
            )

    def get_normalized_wts_file(self):
        
        return os.path.join(
            self._feat_label_dir,
            '{}_wts'.format(self._dataset)
        )

    def get_nb_channels(self):
        
        return self._nb_channels

    def get_nb_classes(self):
        
        return len(self._unique_classes)

    def nb_frames_1s(self):
        
        return self._nb_label_frames_1s

    def get_hop_len_sec(self):
       
        return self._hop_len_s

    def get_nb_frames(self):
        
        return self._max_label_frames

    def get_nb_mel_bins(self):
        
        return self._nb_mel_bins



In [ ]:
# Extracts the features, labels, and normalizes the development and evaluation split features.
if 'dev' in process_str:
    # -------------- Extract features and labels for development set -----------------------------
    dev_feat_cls = FeatureClass(is_eval=False)

    # Extract features and normalize them
    dev_feat_cls.extract_all_feature()
    dev_feat_cls.preprocess_features()

    # # Extract labels in regression mode
    dev_feat_cls.extract_all_labels()

    print("Development dataset extraction finished")

In [ ]:
if 'eval' in process_str:
    # -----------------------------Extract ONLY features for evaluation set-----------------------------
    eval_feat_cls = FeatureClass(is_eval=True)

    # Extract features and normalize them
    eval_feat_cls.extract_all_feature()
    eval_feat_cls.preprocess_features()
    print("Evaluation dataset extraction finished")